In [3]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.optimizers import Adam , rmsprop

C:\Users\Ayaz\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Loading & splitting data

In [4]:
dataframe = pandas.read_csv("housing.csv", delim_whitespace=True, header=None)
dataset = dataframe.values
X = dataset[:,0:13]
Y = dataset[:,13]


In [5]:
mean = X.mean(axis=0)
X -= mean
std = X.std(axis=0)
X /= std


### Creating baseline model 

In [21]:
def baseline_model():
    model=Sequential()
    model.add(Dense(13, activation='relu' ,input_shape=(13,)))
    model.add(Dense(1))
    model.compile(rmsprop(lr=0.002),loss='mean_squared_error')
    return model

In [5]:
seed = 7
numpy.random.seed(seed)
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)


In [6]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: -24.31 (29.78) MSE


### Standardize dataset

In [24]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))


Standardized: -19.51 (21.23) MSE


### Rescaling the output layer with activation function in baseline model

In [25]:
def rescaled_model():
    model=Sequential()
    model.add(Dense(13, activation='relu' ,input_shape=(13,)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(Adam(lr=0.001),loss='mean_squared_error')
    return model

In [26]:
seed = 7
numpy.random.seed(seed)
estimator = KerasRegressor(build_fn=rescaled_model, epochs=50, batch_size=5, verbose=0)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=rescaled_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Rescaled_Model_Result: %.2f (%.2f) MSE" % (results.mean(), results.std()))


Rescaled_Model_Result: -546.51 (276.40) MSE


### Rescaling the baseline model 1

In [20]:
def larger_model():
    model=Sequential()
    model.add(Dense(13, activation='relu' ,input_shape=(13,)))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1))
    model.compile(Adam(lr=0.001),loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

In [21]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))


Larger: -22.63 (28.30) MSE


### Rescaling the baseline model 2

In [22]:
def wider_model():
    model=Sequential()
    model.add(Dense(20, activation='relu' ,input_shape=(13,)))
    model.add(Dense(1))
    model.compile(Adam(lr=0.002),loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

In [23]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Wider: -22.41 (21.21) MSE


### Creating model that overfits

In [6]:
def overfitting_model():
    model=Sequential()
    model.add(Dense(20, activation='relu' ,input_shape=(13,)))
    model.add(Dense(13, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1))
    model.compile(Adam(lr=0.08),loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

In [8]:
seed = 7
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=overfitting_model, epochs=200, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Wider: -67.74 (52.15) MSE


### Retunning the model 

In [13]:
def retuning_model():
    model=Sequential()
    model.add(Dense(39, activation='relu' ,input_shape=(13,)))
    model.add(Dense(13, activation='relu'))
    model.add(Dense(1))
    model.compile(Adam(lr=0.002),loss='mean_squared_error')
    return model

In [10]:
seed = 7
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=retuning_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=20, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Retuning_Result: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Retuning_Result: -22.31 (38.51) MSE


### Creating model with keras functional API 

In [17]:
import keras
from keras import layers

def functional_api():
    inputs=keras.Input(shape=(13,))
    hl_1=layers.Dense(13,activation='relu')(inputs)
    outputs=layers.Dense(1)(hl_1)
    model=keras.Model(inputs,outputs)
    model.compile(optimizer=Adam(lr=0.002),loss='mean_squared_error')
    return model

In [18]:
estimator = KerasRegressor(build_fn=functional_api, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("functional_API_Result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

functional_API_Result: -1234.48% (639.74%)


In [19]:
k = 4
num_val_samples = len(X) // k
num_epochs = 100
all_scores = []

In [ ]:
for i in range(k):
    print('processing fold #', i)
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = Y[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = numpy.concatenate([X[:i * num_val_samples],X[(i + 1) * num_val_samples:]],axis=0)
    partial_train_targets = numpy.concatenate([Y[:i * num_val_samples],Y[(i + 1) * num_val_samples:]],axis=0)
model = baseline_model()
model.fit(partial_train_data, partial_train_targets,epochs=num_epochs, batch_size=1, verbose=0)
val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
all_scores.append(val_mae)
all_scores.append(val_mse)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
